# Trabalho Prático 1: Compressão de Imagens

**Grupo**:

- Isis Ferreira Carvalho (2020006663)
- Lucas Mesquita Andrade (2020054668)
- Victor Augusto de Lima Cruz (2020054854)

## Ideia

Comparar a performance da compressão JPEG utilizando várias transformadas diferentes.

{elaborar}

## Implementação

In [80]:
# Importações
%matplotlib inline
import cv2
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import fftpack
import matplotlib.image as mpimg
from collections import Counter
import heapq
import struct
import json
import warnings
import pywt
import math

warnings.simplefilter(action='ignore', category=FutureWarning)

In [62]:
"""
    Função de leitura de dados e conversão pra escala de cinza
    Retorna: imagem original convertida para grayscale (ou imagem original se ela já está em grayscale)
"""
def ConvertToPgm(filepath, outpath):
    try:
        color_image = cv2.imread(filepath)
        gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
        gray_image_8bit = cv2.convertScaleAbs(gray_image)
    except:
        return cv2.imread(filepath, 0)
    
    if color_image is None:
        print(f"Couldn't read file")

    cv2.imwrite(outpath, gray_image_8bit)
    return gray_image_8bit


In [63]:
img = ConvertToPgm('img/lena512.png', 'img/lena512_gray.pgm')
img

In [64]:
img

### Huffman

In [65]:
""" 
    Lógica da Codificação de Huffman, com métodos de codificação e decodificação 
"""

# Calcula frequências dos tons de cinza 
def calculate_frequencies(arr):
    flattened = arr.flatten()
    return Counter(flattened)

# Classe que implementa o nó da árvore para Codificação de Huffman
class HuffmanNode:
    def __init__(self, value=None, freq=0):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq

# Constrói árvore de Huffman usando heap e pegando elemento mínimo 
def build_huffman_tree(frequencies):
    heap = [HuffmanNode(value, freq) for value, freq in frequencies.items()]
    heapq.heapify(heap)
    while len(heap) > 1:
        node1 = heapq.heappop(heap)
        node2 = heapq.heappop(heap)
        merged = HuffmanNode(freq=node1.freq + node2.freq)
        merged.left = node1
        merged.right = node2
        heapq.heappush(heap, merged)
    return heap[0]

def generate_huffman_codes(node, prefix='', codebook=None):
    if codebook is None:
        codebook = {}
    if node.value is not None:
        codebook[node.value] = prefix
    else:
        generate_huffman_codes(node.left, prefix + '0', codebook)
        generate_huffman_codes(node.right, prefix + '1', codebook)
    return codebook

def encode_array(arr, codebook):
    flattened = arr.flatten()
    encoded_bits = ''.join(codebook[value] for value in flattened)
    return encoded_bits

def save_encoded_data(encoded_bits, codebook, original_shape, file_path):
    with open(file_path, 'wb') as f:
        # Save the original shape
        f.write(struct.pack('>II', *original_shape))
        codebook_converted = {int(k): v for k, v in codebook.items()}
        
        # Save the codebook
        codebook_str = json.dumps(codebook_converted)
        f.write(struct.pack('>I', len(codebook_str)))
        f.write(codebook_str.encode('utf-8'))
        
        # Save the encoded bitstream length
        bitstream_len = len(encoded_bits)
        f.write(struct.pack('>I', bitstream_len))
        
        # Save the encoded bitstream
        byte_array = bytearray()
        for i in range(0, bitstream_len, 8):
            byte = encoded_bits[i:i+8]
            byte_array.append(int(byte.ljust(8, '0'), 2))
        f.write(byte_array)

def load_encoded_data(file_path):
    with open(file_path, 'rb') as f:
        # Read the original shape
        original_shape = struct.unpack('>II', f.read(8))
        
        # Read the codebook
        codebook_len = struct.unpack('>I', f.read(4))[0]
        codebook_str = f.read(codebook_len).decode('utf-8')
        codebook = json.loads(codebook_str)
        
        # Read the encoded bitstream length
        bitstream_len = struct.unpack('>I', f.read(4))[0]
        
        # Read the encoded bitstream
        bitstream_bytes = f.read()
        encoded_bits = ''.join(format(byte, '08b') for byte in bitstream_bytes)[:bitstream_len]
    
    return encoded_bits, codebook, original_shape

def decode_huffman(encoded_bits, codebook):
    reverse_codebook = {v: k for k, v in codebook.items()}
    decoded_values = []
    
    current_bits = ''
    for bit in encoded_bits:
        current_bits += bit
        if current_bits in reverse_codebook:
            decoded_values.append(reverse_codebook[current_bits])
            current_bits = ''
    
    return decoded_values

def bits_to_array(decoded_values, original_shape):
    return np.array(decoded_values).reshape(original_shape)

### JPEG

In [66]:
"""
    Função que faz o padding da imagem caso sua altura ou largura não sejam múltiplos de 8
"""
def pad_image(img, block_size):

    width, height = img.shape[0], img.shape[1]

    if width % 8 != 0 or height % 8 != 0:
        #padding é necessário
        height = np.float32(height) 
        width = np.float32(width) 

        new_width = np.int32(math.ceil(width/block_size) * block_size)
        new_height = np.int32(math.ceil(height/block_size) * block_size)

        padded_img = np.zeros((new_width, new_height))
        padded_img[0:int(width),0:int(height)] = img[0:int(width),0:int(height)]

        width, height = new_width, new_height
        cv2.imwrite('padded_img.bmp', np.uint8(padded_img)) #inserir nome na representação intermediária do file
    else:
        padded_img = img

    return padded_img, width, height

In [67]:
"""
    Funções de aplicação de transformadas e suas inversas
"""
def apply_dct(img):
    return fftpack.dct(fftpack.dct(img.T, norm='ortho').T, norm='ortho')

def apply_idct(coefficients):
    return fftpack.idct(fftpack.idct(coefficients.T, norm='ortho').T, norm='ortho')

In [68]:
"""
    Função de caminhamento em zigzag na matriz pós-transofrmada e quantização, para captar maiores coeficientes primeiro
    {TO-DO: melhorar comentários como se fossem nossos}
"""

def zigzag(input):
    h, v = 0, 0
    vmin, hmin = 0, 0
    vmax = input.shape[0]
    hmax = input.shape[1]
    i = 0

    output = np.zeros((vmax * hmax))

    while ((v < vmax) and (h < hmax)):
        if ((h + v) % 2) == 0:                 # going up
            if (v == vmin):
                output[i] = input[v, h]        # if we got to the first line
                if (h == hmax):
                    v += 1
                else:
                    h += 1
                i += 1
            elif ((h == hmax - 1) and (v < vmax)):   # if we got to the last column
                output[i] = input[v, h]
                v += 1
                i += 1
            elif ((v > vmin) and (h < hmax - 1)):    # all other cases
                output[i] = input[v, h]
                v -= 1
                h += 1
                i += 1
        else:                                    # going down
            if ((v == vmax - 1) and (h <= hmax - 1)):       # if we got to the last line
                output[i] = input[v, h]
                h += 1
                i += 1
            elif (h == hmin):                  # if we got to the first column
                output[i] = input[v, h]
                if (v == vmax - 1):
                    h += 1
                else:
                    v += 1
                i = i + 1
            elif ((v < vmax - 1) and (h > hmin)):     # all other cases
                output[i] = input[v, h]
                v += 1
                h -= 1
                i += 1

        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element
            output[i] = input[v, h]
            break

    return output.astype(int)

def unzigzag(input, vmax, hmax):
    h = 0
    v = 0
    vmin = 0
    hmin = 0
    i = 0
    
    output = np.zeros((vmax, hmax))
    
    while ((v < vmax) and (h < hmax)): 
        if ((h + v) % 2) == 0:                 # going up
            if (v == vmin):
                output[v, h] = input[i]        # if we got to the first line
                if (h == hmax):
                    v = v + 1
                else:
                    h = h + 1                        
                i = i + 1
            elif ((h == hmax -1 ) and (v < vmax)):   # if we got to the last column
                output[v, h] = input[i] 
                v = v + 1
                i = i + 1
            elif ((v > vmin) and (h < hmax -1 )):    # all other cases
                output[v, h] = input[i] 
                v = v - 1
                h = h + 1
                i = i + 1

        else:                                    # going down
            if ((v == vmax -1) and (h <= hmax -1)):       # if we got to the last line
                output[v, h] = input[i] 
                h = h + 1
                i = i + 1
            elif (h == hmin):                  # if we got to the first column
                output[v, h] = input[i] 
                if (v == vmax -1):
                    h = h + 1
                else:
                    v = v + 1
                i = i + 1                
            elif((v < vmax -1) and (h > hmin)):     # all other cases
                output[v, h] = input[i] 
                v = v + 1
                h = h - 1
                i = i + 1

        if ((v == vmax-1) and (h == hmax-1)):          # bottom right element        
            output[v, h] = input[i] 
            break


    return output

In [69]:
"""
    Declaração da matriz de quantização a ser usada na codificação
"""
QM = np.matrix([[16, 11, 10, 16, 24, 40, 51, 61], 
                [12, 12, 14, 19, 26, 58, 60, 55],
                [14, 13, 16, 24, 40, 57, 69, 56],
                [14, 17, 22, 29, 51, 87, 80, 62],
                [18, 22, 37, 56, 68, 109, 103, 77],
                [24, 35, 55, 64, 81, 104, 113, 92],
                [49, 64, 78, 87, 103, 121, 120, 101],
                [72, 92, 95, 98, 112, 100, 103, 99]])

In [70]:
""" 
    {TO-DO: DAR UNS PRINT EM RESULTADOS INTERMEDIÁRIOS, COMENTAR, PENSAR MELHRO NO FLUXO}
"""
def encode_jpeg(img, transformada, quantiz_matrix, encoded_file_path='encoded_image.bin', plot=False):
    img = img.astype(float)
    img = img - 128
    block_size = 8
    padded_img, width, height = pad_image(img, block_size)
    # num_blocks_w, num_blocks_h = width/block_size, height/block_size

    new_img = np.zeros((width, height)).astype(int) # Para guardar representação pós transformada, quantização e zigzag

    for x in range(0, width, block_size):
        for y in range(0, height, block_size):
            block = padded_img[x:x+block_size, y:y+block_size]
            dct = transformada(block)
            dct_quantized = np.round(np.divide(dct, quantiz_matrix)).astype(int)
            dct_zigzag = zigzag(dct_quantized)
            reshaped = np.reshape(dct_zigzag, (block_size, block_size))
            new_img[x:x+block_size, y:y+block_size] = reshaped #dct_quantized

    # HUFFMAN
    arr = np.array(new_img)
    frequencies = calculate_frequencies(arr)
    huffman_tree = build_huffman_tree(frequencies)
    codebook = generate_huffman_codes(huffman_tree)
    encoded_bits = encode_array(arr, codebook)
    save_encoded_data(encoded_bits, codebook, img.shape, encoded_file_path)

    if plot:
        plt.imshow(img+128, cmap = 'gray', clim=(0, 255))
        plt.show()
        plt.imshow(new_img, cmap = 'gray', clim=(0, 255))
        plt.show()

    return new_img

In [71]:
def decode_jpeg(encoded_file_path, transformada, quantiz_matrix, plot=False):
    encoded_bits_read, codebook_read, original_shape_read = load_encoded_data(encoded_file_path)
    (orig_w, orig_h) = original_shape_read

    if orig_w % 8 != 0:
        padded_w = orig_w + 8 - (orig_w % 8)
    else:
        padded_w = orig_w
    if orig_h % 8 != 0:
        padded_h = orig_h + 8 - (orig_h % 8)
    else:
        padded_h = orig_h
            
    codebook_treated = {int(k):v for k,v in codebook_read.items()}
    decoded_values = decode_huffman(encoded_bits_read, codebook_treated)
    reconst_matrix = bits_to_array(decoded_values, (padded_w, padded_h))     # Final da decodificação de Huffman

    reconst_img = np.zeros((padded_w, padded_h)).astype(int)

    block_size = 8
    for x in range(0, padded_w, block_size):
        for y in range(0, padded_h, block_size):
            block = reconst_matrix[x:x+block_size, y:y+block_size]
            block = unzigzag(block.flatten(), block_size, block_size)
            reconst_dequantized = np.multiply(block, quantiz_matrix).astype(int)
            inverse_dct = transformada(reconst_dequantized)
            reconst_img[x:x+block_size, y:y+block_size] = inverse_dct

    reconst_img = reconst_img[:orig_w, :orig_h]
    reconst_img += 128

    reconst_img = np.uint8(reconst_img)

    if plot:
        plt.imshow(reconst_img, cmap = 'gray', clim=(0, 255))

    return reconst_img

In [72]:
# new_img = encode_jpeg(img, apply_dct, QM, plot=True)

AttributeError: 'NoneType' object has no attribute 'astype'

In [ ]:
# reconstructed_img = decode_jpeg('encoded_image.bin', apply_idct, QM, plot=True)

In [ ]:
# reconstructed_img.tofile('testfile.pgm')

# Wavelet Puro

In [114]:
def encodeHuffmanWavelet(img, encoded_file_path, shape):
  arr = np.array(img)
  frequencies = calculate_frequencies(arr)
  huffman_tree = build_huffman_tree(frequencies)
  codebook = generate_huffman_codes(huffman_tree)
  encoded_bits = encode_array(arr, codebook)
  save_encoded_data(encoded_bits, codebook, shape, encoded_file_path)

def decodeHuffmanWavelet(encoded_file_path):
    encoded_bits_read, codebook_read, original_shape_read = load_encoded_data(encoded_file_path)
    (orig_w, orig_h) = original_shape_read

    if orig_w % 8 != 0:
        padded_w = orig_w + 8 - (orig_w % 8)
    else:
        padded_w = orig_w
    if orig_h % 8 != 0:
        padded_h = orig_h + 8 - (orig_h % 8)
    else:
        padded_h = orig_h

    padded_h = int(padded_h / 2)
    padded_w = int(padded_w * 2)

    codebook_treated = {int(k):v for k,v in codebook_read.items()}
    decoded_values = decode_huffman(encoded_bits_read, codebook_treated)
    reconst_matrix = bits_to_array(decoded_values, (padded_w, padded_h))

    return reconst_matrix, original_shape_read

In [115]:
def compressWavelet(img, encoded_file_path, percentile_threshold):
  padded_img, _, _ = pad_image(img, 8)

  coeffs = pywt.dwt2(padded_img, 'haar')
  cA, (cH, cV, cD) = coeffs

  allCoef = np.abs(np.concatenate((cH.flatten(), cV.flatten(), cD.flatten())))
  threshold = np.percentile(allCoef, percentile_threshold)

  cH_thresholded = pywt.threshold(cH, threshold, mode='hard')
  cV_thresholded = pywt.threshold(cV, threshold, mode='hard')
  cD_thresholded = pywt.threshold(cD, threshold, mode='hard')

  cA = cA.astype(int)
  cH_thresholded = cH_thresholded.astype(int)
  cV_thresholded = cV_thresholded.astype(int)
  cD_thresholded = cD_thresholded.astype(int)

  result = np.vstack((cA, cH_thresholded, cV_thresholded, cD_thresholded))

  encodeHuffmanWavelet(result, encoded_file_path, img.shape)

def decompressWavelet(encoded_file_path):
    allComponents, orig_size = decodeHuffmanWavelet(encoded_file_path)

    size = int(allComponents.shape[0]/4)

    cA = allComponents[:size, :]
    cH = allComponents[size:size*2, :]
    cV = allComponents[size*2:size*3, :]
    cD = allComponents[size*3:, :]

    coeffs_thresholded = (cA, (cH, cV, cD))
    img_decompressed = pywt.idwt2(coeffs_thresholded, 'haar')


    img_decompressed = img_decompressed[:orig_size[0], :orig_size[1]]

    return img_decompressed

In [116]:
def compressWaveletAll(img, encoded_file_path, percentile_threshold):
  padded_img, _, _ = pad_image(img, 8)

  coeffs = pywt.dwt2(padded_img, 'haar')
  cA, (cH, cV, cD) = coeffs

  allCoef = np.abs(np.concatenate((cH.flatten(), cV.flatten(), cD.flatten(), cA.flatten())))
  threshold = np.percentile(allCoef, percentile_threshold)

  cA_thresholded = pywt.threshold(cA, threshold, mode='hard')

  cH_thresholded = pywt.threshold(cH, threshold, mode='hard')
  cV_thresholded = pywt.threshold(cV, threshold, mode='hard')
  cD_thresholded = pywt.threshold(cD, threshold, mode='hard')

  cA_thresholded = cA_thresholded.astype(int)
    
  cH_thresholded = cH_thresholded.astype(int)
  cV_thresholded = cV_thresholded.astype(int)
  cD_thresholded = cD_thresholded.astype(int)

  result = np.vstack((cA_thresholded, cH_thresholded, cV_thresholded, cD_thresholded))

  encodeHuffmanWavelet(result, encoded_file_path, img.shape)


In [117]:
def createCompressWaveletWithPercentile(percentile):
    def compress(img, encoded_file_path):
        compressWavelet(img, encoded_file_path, percentile)
    return compress

def createCompressWaveletAllWithPercentile(percentile):
    def compress(img, encoded_file_path):
        compressWaveletAll(img, encoded_file_path, percentile)
    return compress

# Experimentos

In [ ]:
vou organizando o código enquanto isso
MALDITO LZ....
não dá pra entender

In [124]:
# implementação do Professor
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

def psnr(predictions, targets):
    rmsev = rmse(predictions, targets)
    return 20 * np.log10(255/rmsev)

In [78]:
def convertToPgm(filepath, outpath):
    color_image = cv2.imread(filepath)

    if color_image is None:
        print(f"Couldn't read file")

    gray_image = cv2.cvtColor(color_image, cv2.COLOR_BGR2GRAY)
    gray_image_8bit = cv2.convertScaleAbs(gray_image)
    cv2.imwrite(outpath, gray_image_8bit)

In [79]:
def aux_jpeg_encode(img, output_filepath):
    encode_jpeg(img, apply_dct, QM, output_filepath)

def aux_jpeg_decode(output_filepath):
    return decode_jpeg(output_filepath, apply_idct, QM)

In [82]:
def evaluateCompression(compressionFunction, decompressionFunction):
    files = os.listdir('img')
    df = pd.DataFrame()
    for file in files:
        input_filepath = './img/' + file
        output_filepath = './tmp/' + file
        
        imageMatrix = cv2.imread(input_filepath, 0)
        if imageMatrix is None:
            continue

        compressionFunction(imageMatrix, output_filepath)
        decompressedImageMatrix = decompressionFunction(output_filepath)
#         print(f'min: {decompressedImageMatrix.min()}, max: {decompressedImageMatrix.max()}')
        compression_rate = os.path.getsize(input_filepath) / os.path.getsize(output_filepath)
        psnr_imgs = psnr(imageMatrix, decompressedImageMatrix)
        
        df = df.append({'file': file, 
                        'shape': imageMatrix.shape,
                        'result_shape': imageMatrix.shape,
                        'compression_rate': compression_rate,
                        'psnr': psnr_imgs
                       }, ignore_index=True)
        
    return df
    

não era só future warning
que se foda o futuro
kkkkkkkkkkk
vou desativar os future warnings

In [131]:
compare_df = pd.DataFrame()

# JPEG NORMAL

In [132]:
evaluation_df = evaluateCompression(aux_jpeg_encode, aux_jpeg_decode)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'jpeg', 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 36.55 ± 3.86
Compression Rate - 5.07 ± 0.71


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",6.235749,34.836174
1,barbara.pgm,"(512, 512)","(512, 512)",4.757258,34.070900
2,boats.pgm,"(576, 720)","(576, 720)",5.355932,36.301283
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",6.773264,41.803347
4,Brain1.pgm,"(911, 699)","(911, 699)",5.134791,40.388619
5,Brain2.pgm,"(901, 675)","(901, 675)",5.350173,41.372263
6,Brain3.pgm,"(867, 691)","(867, 691)",5.024319,39.784053
7,Brain4.pgm,"(881, 719)","(881, 719)",5.529411,42.816451
8,Brain5.pgm,"(903, 713)","(903, 713)",5.527885,42.861801
9,bridge.pgm,"(512, 512)","(512, 512)",4.103968,31.700034


# WAVELET FILTRANDO DETALHES

In [130]:
# 100 perc
percentile = 100
evaluation_df = evaluateCompression(createCompressWaveletWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'wavelet' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 30.58 ± 4.78
Compression Rate - 2.5 ± 0.27


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",2.982073,34.369869
1,barbara.pgm,"(512, 512)","(512, 512)",2.326453,25.416889
2,boats.pgm,"(576, 720)","(576, 720)",2.507864,29.711833
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",3.127873,40.537825
4,Brain1.pgm,"(911, 699)","(911, 699)",2.613932,31.149341
5,Brain2.pgm,"(901, 675)","(901, 675)",2.737972,33.648557
6,Brain3.pgm,"(867, 691)","(867, 691)",2.578533,31.167884
7,Brain4.pgm,"(881, 719)","(881, 719)",2.663615,36.017064
8,Brain5.pgm,"(903, 713)","(903, 713)",2.679059,36.844850
9,bridge.pgm,"(512, 512)","(512, 512)",2.400824,25.680811


In [133]:
# 99 perc
percentile = 99
evaluation_df = evaluateCompression(createCompressWaveletWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'wavelet' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 33.21 ± 5.8
Compression Rate - 2.4 ± 0.27


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",2.887214,35.489076
1,barbara.pgm,"(512, 512)","(512, 512)",2.232894,26.935879
2,boats.pgm,"(576, 720)","(576, 720)",2.411692,33.040420
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",3.014008,44.390840
4,Brain1.pgm,"(911, 699)","(911, 699)",2.504619,34.900217
5,Brain2.pgm,"(901, 675)","(901, 675)",2.634907,36.968862
6,Brain3.pgm,"(867, 691)","(867, 691)",2.487575,34.110762
7,Brain4.pgm,"(881, 719)","(881, 719)",2.581480,38.314787
8,Brain5.pgm,"(903, 713)","(903, 713)",2.602375,38.729582
9,bridge.pgm,"(512, 512)","(512, 512)",2.303216,26.955107


In [134]:
# 95 perc
percentile = 95
evaluation_df = evaluateCompression(createCompressWaveletWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'wavelet' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 36.28 ± 5.87
Compression Rate - 2.2 ± 0.25


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",2.637046,37.189797
1,barbara.pgm,"(512, 512)","(512, 512)",2.049642,29.950639
2,boats.pgm,"(576, 720)","(576, 720)",2.215339,37.035468
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",2.775091,46.391719
4,Brain1.pgm,"(911, 699)","(911, 699)",2.306818,39.491759
5,Brain2.pgm,"(901, 675)","(901, 675)",2.404945,41.618351
6,Brain3.pgm,"(867, 691)","(867, 691)",2.293059,38.166470
7,Brain4.pgm,"(881, 719)","(881, 719)",2.387263,42.528660
8,Brain5.pgm,"(903, 713)","(903, 713)",2.409232,42.310451
9,bridge.pgm,"(512, 512)","(512, 512)",2.113706,29.119478


In [135]:
# 90 perc
percentile = 90
evaluation_df = evaluateCompression(createCompressWaveletWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'wavelet' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 38.64 ± 5.94
Compression Rate - 2.04 ± 0.23


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",2.422266,38.568951
1,barbara.pgm,"(512, 512)","(512, 512)",1.876280,32.807953
2,boats.pgm,"(576, 720)","(576, 720)",2.029963,40.126849
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",2.577817,48.023156
4,Brain1.pgm,"(911, 699)","(911, 699)",2.145130,42.418837
5,Brain2.pgm,"(901, 675)","(901, 675)",2.223814,44.458928
6,Brain3.pgm,"(867, 691)","(867, 691)",2.133049,41.070620
7,Brain4.pgm,"(881, 719)","(881, 719)",2.214738,45.888007
8,Brain5.pgm,"(903, 713)","(903, 713)",2.238809,45.368133
9,bridge.pgm,"(512, 512)","(512, 512)",1.951049,31.029211


In [136]:
# 80 perc
percentile = 80
evaluation_df = evaluateCompression(createCompressWaveletWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'wavelet' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 42.29 ± 6.1
Compression Rate - 1.81 ± 0.21


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",2.131065,40.908194
1,barbara.pgm,"(512, 512)","(512, 512)",1.635448,37.447270
2,boats.pgm,"(576, 720)","(576, 720)",1.788860,44.096415
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",2.303576,50.548171
4,Brain1.pgm,"(911, 699)","(911, 699)",1.901209,46.957482
5,Brain2.pgm,"(901, 675)","(901, 675)",1.978649,49.016045
6,Brain3.pgm,"(867, 691)","(867, 691)",1.890093,45.858495
7,Brain4.pgm,"(881, 719)","(881, 719)",1.991002,50.713355
8,Brain5.pgm,"(903, 713)","(903, 713)",2.007586,50.432141
9,bridge.pgm,"(512, 512)","(512, 512)",1.717413,34.216530


# WAVELET FILTRANDO TUDO

In [137]:
# 99 perc
percentile = 99
evaluation_df = evaluateCompression(createCompressWaveletAllWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'waveletAll' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 7.68 ± 3.65
Compression Rate - 7.3 ± 0.16


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",7.304324,18.018276
1,barbara.pgm,"(512, 512)","(512, 512)",7.330277,6.404702
2,boats.pgm,"(576, 720)","(576, 720)",7.327233,6.241424
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",7.156148,19.964062
4,Brain1.pgm,"(911, 699)","(911, 699)",7.466345,8.835617
5,Brain2.pgm,"(901, 675)","(901, 675)",7.181130,8.964418
6,Brain3.pgm,"(867, 691)","(867, 691)",7.257316,8.629680
7,Brain4.pgm,"(881, 719)","(881, 719)",7.405190,8.976227
8,Brain5.pgm,"(903, 713)","(903, 713)",7.215278,9.475918
9,bridge.pgm,"(512, 512)","(512, 512)",7.422206,6.751429


In [138]:
# 80 perc
percentile = 80
evaluation_df = evaluateCompression(createCompressWaveletAllWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'waveletAll' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 24.67 ± 8.74
Compression Rate - 2.89 ± 0.28


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",3.443448,29.048723
1,barbara.pgm,"(512, 512)","(512, 512)",2.751238,20.662695
2,boats.pgm,"(576, 720)","(576, 720)",3.053643,22.447053
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",3.550975,31.066902
4,Brain1.pgm,"(911, 699)","(911, 699)",2.706562,35.077759
5,Brain2.pgm,"(901, 675)","(901, 675)",2.733873,40.508514
6,Brain3.pgm,"(867, 691)","(867, 691)",2.673051,35.402206
7,Brain4.pgm,"(881, 719)","(881, 719)",2.752094,40.363599
8,Brain5.pgm,"(903, 713)","(903, 713)",2.761690,40.729206
9,bridge.pgm,"(512, 512)","(512, 512)",2.875496,19.826312


In [139]:
# 75 perc
percentile = 75
evaluation_df = evaluateCompression(createCompressWaveletAllWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'waveletAll' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 33.97 ± 7.02
Compression Rate - 2.41 ± 0.22


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",2.973776,34.648786
1,barbara.pgm,"(512, 512)","(512, 512)",2.291540,26.235652
2,boats.pgm,"(576, 720)","(576, 720)",2.464587,32.442642
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",3.086368,43.353097
4,Brain1.pgm,"(911, 699)","(911, 699)",2.444718,37.954654
5,Brain2.pgm,"(901, 675)","(901, 675)",2.430324,43.925541
6,Brain3.pgm,"(867, 691)","(867, 691)",2.408622,37.812547
7,Brain4.pgm,"(881, 719)","(881, 719)",2.478014,43.403991
8,Brain5.pgm,"(903, 713)","(903, 713)",2.466536,43.751407
9,bridge.pgm,"(512, 512)","(512, 512)",2.368979,26.508795


In [140]:
# 0 perc
percentile = 0
evaluation_df = evaluateCompression(createCompressWaveletAllWithPercentile(percentile), decompressWavelet)
cur_psnr = str(round(evaluation_df['psnr'].mean(), 2)) + " ± " + str(round(evaluation_df['psnr'].std(), 2))
cur_cr = str(round(evaluation_df['compression_rate'].mean(), 2)) + " ± " + str(round(evaluation_df['compression_rate'].std(), 2))
compare_df = compare_df.append({'name': 'waveletAll' + str(percentile), 
                    'psnr': cur_psnr,
                    'cr': cur_cr,
                   }, ignore_index=True)
print("PSNR -", cur_psnr)
print("Compression Rate -", cur_cr)
evaluation_df

PSNR - 54.94 ± 0.87
Compression Rate - 1.33 ± 0.26


,file,shape,result_shape,compression_rate,psnr
0,airplaneU2.pgm,"(1024, 1024)","(1024, 1024)",1.396581,54.958498
1,barbara.pgm,"(512, 512)","(512, 512)",1.107594,54.339373
2,boats.pgm,"(576, 720)","(576, 720)",1.322372,54.239374
3,bone_scint.pgm,"(1865, 1129)","(1865, 1129)",1.873999,55.261689
4,Brain1.pgm,"(911, 699)","(911, 699)",1.504164,55.957047
5,Brain2.pgm,"(901, 675)","(901, 675)",1.657902,56.086615
6,Brain3.pgm,"(867, 691)","(867, 691)",1.469228,55.963000
7,Brain4.pgm,"(881, 719)","(881, 719)",1.702213,56.553478
8,Brain5.pgm,"(903, 713)","(903, 713)",1.708177,56.530853
9,bridge.pgm,"(512, 512)","(512, 512)",1.097060,55.047464


In [141]:
compare_df

,name,psnr,cr
0,jpeg,36.55 ± 3.86,5.07 ± 0.71
1,wavelet99,33.21 ± 5.8,2.4 ± 0.27
2,wavelet95,36.28 ± 5.87,2.2 ± 0.25
3,wavelet90,38.64 ± 5.94,2.04 ± 0.23
4,wavelet80,42.29 ± 6.1,1.81 ± 0.21
5,waveletAll99,7.68 ± 3.65,7.3 ± 0.16
6,waveletAll80,24.67 ± 8.74,2.89 ± 0.28
7,waveletAll75,33.97 ± 7.02,2.41 ± 0.22
8,waveletAll0,54.94 ± 0.87,1.33 ± 0.26


### Comprimento de corrida

In [ ]:
new_img.astype('uint8').tofile('test_file.txt')

In [ ]:
# contém dúvida

In [ ]:
def get_run_length_encoding(image):
    i = 0
    skip = 0
    stream = []    
    bitstream = ""
    image = image.astype(int)
    while i < image.shape[0]:
        if image[i] != 0:            
            stream.append((image[i],skip))
            bitstream = bitstream + str(image[i])+ " " +str(skip)+ " "
            skip = 0
        else:
            skip = skip + 1
        i = i + 1

    return bitstream

In [ ]:
arranged = new_img.flatten()

# Now RLE encoded data is written to a text file (You can check no of bytes in text file is very less than no of bytes in the image
# THIS IS COMPRESSION WE WANTED, NOTE THAT ITS JUST COMPRESSION DUE TO RLE, YOU CAN COMPRESS IT FURTHER USING HUFFMAN CODES OR MAY BE 
# REDUCING MORE FREQUENCY COEFFICIENTS TO ZERO)

bitstream = get_run_length_encoding(arranged)

In [ ]:
bitstream

In [ ]:
new_img[0][0:10]

In [ ]:
bitstream = str(new_img.shape[0]) + " " + str(new_img.shape[1]) + " " + bitstream + ";"

# Written to image.txt
file1 = open("image.txt","w")
file1.write(bitstream)
file1.close()